In [3]:
import torch
import torch.nn as nn 
import os 
import matplotlib.pyplot as plt

FILE_PATH = os.getcwd()
print(FILE_PATH)


/Users/mohitdulani/Desktop/personal/sencillo-robotics


https://arxiv.org/pdf/2504.05299 -> amazing vlm paper

![SmolVLM](assets/images/smolvlm.png)

---
The first half is same as what we did for ViT paper ! just that the backbon is of CLIP model


CLIP outputs 512 dim features 
and so does the its text embeddings 

---

But we are gonna use the SigLIP instead of CLIP lets make the architecture !  

So, lets start with it , the concatenation of the vision and text tokens and passing to a LM model 


Vision tokens : pass it to SigLIP image backbone , then get reshape the `last_hidden_state` to the H x W x C and then try to reduce the no. of tokens using pixel shuffling , once tokens are reduced pass that to the linear layer and the output from the linear layer to any GPT based LM (language model) model and you get your output token stream !! 



### Vision encoder
this is also taken from the siglip model 


Outputs : 
1. last hidden state
2. pooler output 

In [1]:
from transformers import SiglipVisionConfig, SiglipVisionModel

# Initializing a SiglipVisionConfig with google/siglip-base-patch16-224 style configuration
configuration = SiglipVisionConfig()

# Initializing a SiglipVisionModel (with random weights) from the google/siglip-base-patch16-224 style configuration
model = SiglipVisionModel(configuration)

print(model)

SiglipVisionModel(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(196, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attn): SiglipAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, 

In [6]:
image = torch.randn(size= (3, 224, 224))
image.shape

torch.Size([3, 224, 224])

In [10]:
batch_size = 1
num_patches = 196   # 14x14
patch_dim = 768
llm_dim = 1024
grid_size = 14
vit_dim = patch_dim
upscale_factor = 2  # e.g., merge 2x2 patches into one super-patch

In [ ]:
# get the output features 
image = torch.randn(size= (1 ,3, 224, 224))

# Vision model 
patch_embeddings = model(image).last_hidden_state

# --- Step 2: Reshape to grid for PixelShuffle ---
grid_size = int(num_patches ** 0.5)  # 14
x = patch_embeddings.view(batch_size, grid_size, grid_size, vit_dim)  # [B, 14, 14, 768]
x = x.permute(0, 3, 1, 2)  # [B, 768, 14, 14]

# --- Step 3: Pixel Shuffle ---
out_channels = vit_dim // (upscale_factor**2)  # 768 / 4 = 192
x = x[:, :out_channels * (upscale_factor**2), :, :]  # ensure divisible
pixel_shuffle = nn.PixelShuffle(upscale_factor)
x = pixel_shuffle(x)  # [B, 192, 28, 28]

# --- Step 4: Flatten back to sequence ---
B, C, H, W = x.shape
x = x.permute(0, 2, 3, 1).reshape(B, H*W, C)  # [B, 784, 192]

# --- Step 5: Linear projection to LLM embedding size ---
linear_proj = nn.Linear(C, llm_dim)
vision_tokens_for_lm = linear_proj(x)  # [B, 784, 1024]


In [ ]:
print(vision_tokens_for_lm.shape)

torch.Size([1, 784, 1024])


### Text Encoder

this is also taken from the SigLIP model 

In [ ]:
from transformers import SiglipTextConfig , SiglipTextModel

config = SiglipTextConfig()
model = SiglipTextModel(config)
tokenizer = 

print(model)